In [1]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install yfinance yahoo_fin

In [2]:
import yfinance as yf

# Get the stock data for Apple Inc.
stock = yf.Ticker("AAPL")

# Access the financials data
financials = stock.financials

# Print the financials data
# print(financials["2022-09-30"])

print(stock.info)

{'address1': 'One Apple Park Way', 'city': 'Cupertino', 'state': 'CA', 'zip': '95014', 'country': 'United States', 'phone': '408 996 1010', 'website': 'https://www.apple.com', 'industry': 'Consumer Electronics', 'sector': 'Technology', 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts. In addition, the company offers various services, such as Apple Arcade, a game subscription service; Apple Fitness+, a personalized fitness 

In [3]:
import pandas as pd
from yahoo_fin import stock_info as si
df1 = pd.DataFrame( si.tickers_sp500() )
df2 = pd.DataFrame( si.tickers_nasdaq() )
df3 = pd.DataFrame( si.tickers_dow() )

In [4]:
sym1 = set( symbol for symbol in df1[0].values.tolist() )
sym2 = set( symbol for symbol in df2[0].values.tolist() )
sym3 = set( symbol for symbol in df3[0].values.tolist() )
symbols = list(set.union( sym1, sym2, sym3))
print(symbols[:5])
print(len(symbols))


['KRYS', 'ASCA', 'DPRO', 'SSBK', 'KBWD']
5148


In [18]:
import yfinance as yf
def is_numeric(value):
    return isinstance(value, (int, float))

def pick_stocks(ticker, fundamental_indicators, technical_indicators, picked_stocks):
    # Use the `Ticker` function to retrieve stock data for the specified ticker
    ticker_data = yf.Ticker(ticker)

    # Use the `info` method to retrieve fundamental data for the stock
    stock_info = ticker_data.info
    if not stock_info:
        return False
    
    # Check if the stock passes the fundamental indicators
    if not stock_info.get('trailingPE', 0) or not is_numeric(stock_info['trailingPE']) or stock_info['trailingPE'] > fundamental_indicators['pe_ratio_max']:
        return False
    if not stock_info.get('priceToBook', 0) or stock_info['priceToBook'] > fundamental_indicators['pb_ratio_max']:
        return False
    if not stock_info.get('returnOnAssets', 0) or stock_info['returnOnAssets'] < fundamental_indicators['return_on_assets_min']:
        return False
    # if not stock_info.get('returnOnEquity', 0) or stock_info['returnOnEquity'] < fundamental_indicators['return_on_equity_min']:
    #    return False
    #if not stock_info.get('debtToEquity', 0) or stock_info['debtToEquity'] > fundamental_indicators['debt_to_equity_max']:
    #    return False
    
    print(f"{ticker} Passed first 3 fundamental filters.....")
    
    #print(stock_info.get('currentRatio', 0))
    
    if not stock_info.get('currentRatio', 0) or stock_info['currentRatio'] < fundamental_indicators['current_ratio_min']:
        return False
    
    freeCashFlow = stock_info.get('freeCashflow', 0)
    totalMarketValue = stock_info.get('regularMarketPreviousClose', 0) * stock_info.get('sharesOutstanding', 0)

    #print(freeCashFlow, totalMarketValue)
    
    pcf = totalMarketValue / freeCashFlow if freeCashFlow else None
    if not pcf or pcf > fundamental_indicators['price_to_free_cash_flow_max']:
        return False
    
    
    totalDebt = stock_info.get('totalDebt', 0)
    #print(totalDebt)
    debtToMarketValue = totalDebt / totalMarketValue if totalMarketValue else None
    if not debtToMarketValue or debtToMarketValue > fundamental_indicators['debt_to_equity_max']:
        return False
    
    
    print(f"{ticker} Passed all fundamental filters.....")
    
    # Use the `history` method to retrieve technical data for the stock
    stock_data = ticker_data.history(period="200d", interval="1d")

    # Check if the stock passes the technical indicators
    # Calculate the 50-day and 200-day moving averages
    ma50 = stock_data['Close'][-50:].mean()
    ma200 = stock_data['Close'][-200:].mean()

    # Select the stocks where the 50-day moving average is above the 200-day moving average
    if ma50 <= ma200:
        return False

    # If the stock passes all the indicators, add it to the list of picked stocks
    picked_stocks.append(ticker_data)
    print("Found one good stock!")
    return True

# Set the list of stock tickers to consider
stock_tickers = symbols
# Set the number of stocks to pick
num_stocks = 20

# Set the fundamental indicators
fundamental_indicators = {
    'pe_ratio_max': 20,
    'pb_ratio_max': 5,
    'return_on_assets_min': 0.1,
     #'return_on_equity_min': 0.05,
    'debt_to_equity_max': 1.0,
    'current_ratio_min': 0.5,
    'price_to_free_cash_flow_max': 30
}

# Set the technical indicators
technical_indicators = {
    'average_price_max': 100,
    'average_volume_min': 10000
}

# Initialize an empty list to store the picked stocks
picked_stocks = []
i = 0

import concurrent.futures
from tqdm import tqdm
import time

MAX_RETRIES = 5
INITIAL_DELAY = 5  # in seconds
THREAD_LIMIT = 5

def process_stock(ticker):
    retries = 0
    delay = INITIAL_DELAY
    while retries < MAX_RETRIES:
        try:
            pick_stocks(ticker, fundamental_indicators, technical_indicators, picked_stocks)
            return ticker
        except Exception as e:  # Replace this with the specific exception for 'too many requests' if known
            if "404 client error" in str(e).lower():  # Check if the exception message contains 'too many requests'
                retries += 1
                time.sleep(delay)
                delay *= 2  # Double the delay for the next retry
            else:
                print(e)  # If it's another exception, raise it
    return None  # Return None if all retries fail

# Assuming you have these variables defined somewhere in your code
# stock_tickers = [...]
# fundamental_indicators = {...}
# technical_indicators = {...}
# picked_stocks = [...]


# Using ThreadPoolExecutor to parallelize the process
with concurrent.futures.ThreadPoolExecutor(max_workers=THREAD_LIMIT) as executor:
    for _ in tqdm(executor.map(process_stock, stock_tickers), total=len(stock_tickers), desc="Processing stocks"):
        continue

# Print the list of picked stocks
print(picked_stocks)


Processing stocks:   1%|▌                                                  | 56/5148 [00:02<03:09, 26.91it/s]

PLPC Passed first 3 fundamental filters.....
OCUP Passed first 3 fundamental filters.....
UTMD Passed first 3 fundamental filters.....
UTMD Passed all fundamental filters.....


Processing stocks:   1%|▌                                                  | 56/5148 [00:19<03:09, 26.91it/s]

GSM Passed first 3 fundamental filters.....
GSM Passed all fundamental filters.....
Found one good stock!
FANG Passed first 3 fundamental filters.....
PSHG Passed first 3 fundamental filters.....
BWMX Passed first 3 fundamental filters.....
STLD Passed first 3 fundamental filters.....
STLD Passed all fundamental filters.....
LEGH Passed first 3 fundamental filters.....
LEGH Passed all fundamental filters.....
Found one good stock!
PNT Passed first 3 fundamental filters.....
PNT Passed all fundamental filters.....
Found one good stock!
MCRI Passed first 3 fundamental filters.....
MCRI Passed all fundamental filters.....
JFIN Passed first 3 fundamental filters.....
VYGR Passed first 3 fundamental filters.....
OMAB Passed first 3 fundamental filters.....
CPRX Passed first 3 fundamental filters.....
CPRX Passed all fundamental filters.....
TXMD Passed first 3 fundamental filters.....
TXMD Passed all fundamental filters.....
RMR Passed first 3 fundamental filters.....
RMR Passed all fundame

Processing stocks:   1%|▌                                               | 58/5148 [02:37<16:23:15, 11.59s/it]

...

DHIL Passed first 3 fundamental filters.....
HDSN Passed first 3 fundamental filters.....
HDSN Passed all fundamental filters.....
Found one good stock!


Processing stocks:  83%|████████████████████████████████████████▌        | 4261/5148 [05:43<01:38,  9.03it/s]

HTTPSConnectionPool(host='query1.finance.yahoo.com', port=443): Max retries exceeded with url: /ws/fundamentals-timeseries/v1/finance/timeseries/LEGR?symbol=LEGR&type=trailingPegRatio&period1=1680134400&period2=1695945600 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f3b2074f280>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))
LPRO Passed first 3 fundamental filters.....
LPRO Passed all fundamental filters.....
Found one good stock!
SMLR Passed first 3 fundamental filters.....
SMLR Passed all fundamental filters.....
WIRE Passed first 3 fundamental filters.....
PFIE Passed first 3 fundamental filters.....
PFIE Passed all fundamental filters.....
Found one good stock!
USLM Passed first 3 fundamental filters.....
MBUU Passed first 3 fundamental filters.....
MBUU Passed all fundamental filters.....
METC Passed first 3 fundamental filters.....
METC Passed all fundamental filters.....
NMIH Passed first 3 fundamental

Processing stocks: 100%|█████████████████████████████████████████████████| 5148/5148 [08:12<00:00, 10.45it/s]

[yfinance.Ticker object <GSM>, yfinance.Ticker object <LEGH>, yfinance.Ticker object <PNT>, yfinance.Ticker object <INMD>, yfinance.Ticker object <BELFB>, yfinance.Ticker object <ITRN>, yfinance.Ticker object <TROW>, yfinance.Ticker object <SCYX>, yfinance.Ticker object <WEYS>, yfinance.Ticker object <ARLP>, yfinance.Ticker object <CVCO>, yfinance.Ticker object <HNRG>, yfinance.Ticker object <ARHS>, yfinance.Ticker object <PLAB>, yfinance.Ticker object <BELFA>, yfinance.Ticker object <CHK>, yfinance.Ticker object <EPSN>, yfinance.Ticker object <CHCI>, yfinance.Ticker object <ESEA>, yfinance.Ticker object <JBSS>, yfinance.Ticker object <CTSH>, yfinance.Ticker object <CHRD>, yfinance.Ticker object <UFPI>, yfinance.Ticker object <BRY>, yfinance.Ticker object <HDSN>, yfinance.Ticker object <LPRO>, yfinance.Ticker object <PFIE>, yfinance.Ticker object <COKE>]


In [19]:
for t in picked_stocks:
    print(t.info['symbol'] + ":" + t.info['shortName'])

GSM:Ferroglobe PLC
LEGH:Legacy Housing Corporation
PNT:POINT Biopharma Global Inc.
INMD:InMode Ltd.
BELFB:Bel Fuse Inc.
ITRN:Ituran Location and Control Ltd
TROW:T. Rowe Price Group, Inc.
SCYX:SCYNEXIS, Inc.
WEYS:Weyco Group, Inc.
ARLP:Alliance Resource Partners, L.P
CVCO:Cavco Industries, Inc.
HNRG:Hallador Energy Company
ARHS:Arhaus, Inc.
PLAB:Photronics, Inc.
BELFA:Bel Fuse Inc.
CHK:Chesapeake Energy Corporation
EPSN:Epsilon Energy Ltd.
CHCI:Comstock Holding Companies, Inc
ESEA:Euroseas Ltd.
JBSS:John B. Sanfilippo & Son, Inc.
CTSH:Cognizant Technology Solutions 
CHRD:Chord Energy Corporation
UFPI:UFP Industries, Inc.
BRY:Berry Corporation (bry)
HDSN:Hudson Technologies, Inc.
LPRO:Open Lending Corporation
PFIE:Profire Energy, Inc.
COKE:Coca-Cola Consolidated, Inc.


In [87]:
import yfinance as yf
import pandas as pd

# Set the ticker for the stock you want to analyze
ticker = "AAPL"

# Retrieve the stock data from yfinance
stock_data = yf.Ticker(ticker).history(period="200d")

# Calculate the 50-day and 200-day moving averages
ma50 = stock_data['Close'][-50:].mean()
ma200 = stock_data['Close'][-200:].mean()

print(ma50)
print(ma200)

141.31402511596679
150.51065853118897
